# Descarga de datos

In [10]:
import datetime, requests, os

def descargar_datos():
    URL_BASE = \
        'https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=observaciones/' 

    fecha_actual = datetime.date.today()
    un_dia       = datetime.timedelta(days=1)

    ANIO_INICIO = fecha_actual.year - 1
    MES_INICIO  = fecha_actual.month
    DIA_INICIO  = fecha_actual.day 

    fecha_inicio = datetime.date(ANIO_INICIO, MES_INICIO, DIA_INICIO)
    
    f = open('datasets/datos.csv', 'w')

    f.write('fecha,hora,temp_c,hum_per,pnm_hpa,dd_gr,ff_kmh,nombre\n')

    while fecha_inicio < fecha_actual:
        r = requests.get(
            URL_BASE + 
            f'datohorario{fecha_inicio.year}{fecha_inicio.month:02}{fecha_inicio.day:02}.txt'
        )

        for line in r.iter_lines(decode_unicode='utf8'):
            if line.startswith((' ', 'F', 'o')):
                continue
                
            dato = ','.join(f'{line}\n'.split(maxsplit=7))
            
            if dato.count(',') != 7:
                continue
            
            f.write(dato)

        fecha_inicio += un_dia

    f.close()
    
if not os.path.exists('datasets/datos.csv'):
    descargar_datos()

# Limpieza de datos

In [11]:
import pandas as pd

In [12]:
datos = pd.read_csv('datasets/datos.csv')

/tmp/ipykernel_117646/1047190064.py:1: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  datos = pd.read_csv('datasets/datos.csv')


In [13]:
datos_filtrados = datos[~datos.ff_kmh.astype(str).str.isalpha()]

In [15]:
datos_filtrados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 769505 entries, 0 to 771499
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   fecha    769505 non-null  int64 
 1   hora     769505 non-null  int64 
 2   temp_c   769505 non-null  object
 3   hum_per  769505 non-null  object
 4   pnm_hpa  769505 non-null  object
 5   dd_gr    769505 non-null  object
 6   ff_kmh   769505 non-null  object
 7   nombre   769505 non-null  object
dtypes: int64(2), object(6)
memory usage: 52.8+ MB


In [16]:
datos_filtrados.head()

,fecha,hora,temp_c,hum_per,pnm_hpa,dd_gr,ff_kmh,nombre
0,7062022,0,10.2,100.0,1018.0,0,0,AEROPARQUE AERO ...
1,7062022,1,9.9,100.0,1017.3,260,2,AEROPARQUE AERO ...
2,7062022,2,9.2,100.0,1016.9,0,0,AEROPARQUE AERO ...
3,7062022,3,9.7,100.0,1016.8,300,6,AEROPARQUE AERO ...
4,7062022,4,9.4,100.0,1016.6,330,11,AEROPARQUE AERO ...


In [17]:
datos_filtrados.tail()

,fecha,hora,temp_c,hum_per,pnm_hpa,dd_gr,ff_kmh,nombre
771495,6062023,19,18.0,72.0,1011.1,0,0,VILLA REYNOLDS AERO ...
771496,6062023,20,17.5,74.0,1012.6,320,4,VILLA REYNOLDS AERO ...
771497,6062023,21,16.0,74.0,1014.7,270,4,VILLA REYNOLDS AERO ...
771498,6062023,22,15.0,81.0,1015.0,230,9,VILLA REYNOLDS AERO ...
771499,6062023,23,15.8,76.0,1015.6,140,22,VILLA REYNOLDS AERO ...


# Exploración de datos